In [1]:
from pydriller import Git
from git import Repo
import os
import re
from pydriller import ModificationType
import json

**WHEN CVV IS MENTIONED, WE MEANT VCC**

Note: This notebook file is only to test if the SZZ algo that is implemented is working component per component, this means getting fix files, blame deleted lines, and blame contextual lines. There's a part where it takes all vcc for every repo and every commit, but **it's not** the one that is used at the end. It is just there to show quickly what is done. The actual execution of this is in the other files. There's also repo cloning here.

## Getting Repo and commit

In [9]:
repo_url_test = "https://github.com/nextcloud/server.git"
commit_hash_test = "94975af6db1551c2d23136c2ea22866a5b416070"
local_repo_path_test = "/repo/nextcloud"

In [11]:
#only run this if cloning is necessary!
#Repo.clone_from(repo_url_test, local_repo_path_test)

<git.repo.base.Repo 'C:\\repo\\nextcloud\\.git'>

In [12]:
repo = Git("C:\\repo\\nextcloud\\.git")

In [14]:
commit = repo.get_commit(commit_hash_test)
print(commit.modified_files)

[<pydriller.domain.commit.ModifiedFile object at 0x000001D43120C610>, <pydriller.domain.commit.ModifiedFile object at 0x000001D43120C340>]


In [15]:
#just a test to check things out
author_name = commit.author.name
commit_message = commit.msg
commit_date = commit.committer_date

# Printing nicely
print("Author:", author_name)
print("Commit Message:", commit_message)
print("Commit Date:", commit_date)

Author: Lukas Reschke
Commit Message: [stable9] Set content-type to "application/octet-stream"

Some browsers such as Firefox on Microsoft Windows otherwise do offer to open the file directly which is kinda silly.

Backport of https://github.com/nextcloud/server/pull/258
Commit Date: 2016-06-30 13:04:54+02:00


## Getting Fix Files

In [8]:
def get_file_name(file):
    file_path = file.old_path
    # If this happens, then the file has been created in this commit
    if file_path == None:
        file_path = file.new_path
    return file_path
def is_file_valid(file_path):
    # Extension check
    file_extension = os.path.splitext(file_path)[1]
    if file_extension in invalid_extensions:
        return False
    # Excluded files check
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    if re.match(exclusions, file_name, re.IGNORECASE):
        return False
    # Test files
    file_dir = '/' + os.path.dirname(file_path) + '/'
    directory_match = re.match(r"^.*\/tests?\/.*$", file_dir, re.IGNORECASE)
    prefix_match = re.match(r"^test.+", file_name, re.IGNORECASE)
    postfix_match = re.match(r".+test$", file_name, re.IGNORECASE)
    return not (directory_match or prefix_match or postfix_match)

In [28]:
#getting the fixed files and validate them
invalid_extensions = ('.txt', '.md', '.man', '.lang', '.loc', '.tex', '.texi', '.rst',
'.gif', '.png', '.jpg', '.jpeg', '.svg', '.ico',
'.css', '.scss', '.less',
'.gradle', '.ini',
'.zip',
'.pdf')
exclusions = r"^(install|changelog(s)?|change(s)?|author(s)?|news|readme|todo|about(s)?|credit(s)?|license|release(s)?|release(s)?|release(_|-)note(s)?|version(s)?|makefile|pom|\.git.*|\.travis|\.classpath|\.project)$"

fix_files = []

for mod_file in commit.modified_files:
    file_path = get_file_name(mod_file)
    #print(file_path)
    if not is_file_valid(file_path):
        continue
    fix_files.append(file_path)
print(fix_files)  

['settings\\controller\\logsettingscontroller.php']


In [16]:
invalid_extensions = ('.txt', '.md', '.man', '.lang', '.loc', '.tex', '.texi', '.rst',
'.gif', '.png', '.jpg', '.jpeg', '.svg', '.ico',
'.css', '.scss', '.less',
'.gradle', '.ini',
'.zip',
'.pdf')
exclusions = r"^(install|changelog(s)?|change(s)?|author(s)?|news|readme|todo|about(s)?|credit(s)?|license|release(s)?|release(s)?|release(_|-)note(s)?|version(s)?|makefile|pom|\.git.*|\.travis|\.classpath|\.project)$"
def get_fix_files(commit):
    fix_files = []
    for mod_file in commit.modified_files:
        file_path = get_file_name(mod_file)
        #print(file_path)
        if not is_file_valid(file_path):
            continue
        fix_files.append(file_path)
    return fix_files

## Blaming deleted lines (test)

In [13]:
def print_blames(blamed):
    all_blamed_hashes = {blame for blames in blamed.values() for blame in blames}
    print("*** Blamed {} commit(s)".format(len(all_blamed_hashes)))
    
def blame_deleted_lines(git_repo, fix_commit, fix_files):
    print("** Blaming the deleted lines...")
    blames = git_repo.get_commits_last_modified_lines(fix_commit)
    blamed_from_deleted_lines = {}
    for file, blamed_hashes in blames.items():
        if file not in fix_files:
            continue
        blamed_from_deleted_lines[file] = blamed_hashes
    print_blames(blamed_from_deleted_lines)
    return blamed_from_deleted_lines

In [30]:
blame_deleted_lines(repo, commit, fix_files)

** Blaming the deleted lines...
*** Blamed 1 commit(s)


{'settings\\controller\\logsettingscontroller.php': {'b5545932e7efec24f36fae76ffe8924e31e1d55a'}}

## Blaming context lines (test)

### Hunks and its validations

In [10]:
def is_useless_line(line):
    return not line or \
            line.startswith("//") or \
            line.startswith("/*") or \
            line.startswith("*") or \
            line.startswith("#") or \
            line.startswith("'''") or \
            line.startswith('"""') or \
            line.startswith("=begin") or \
            line.startswith("<#") or \
            line.startswith("--") or \
            line.startswith("{-") or \
            line.startswith("--[[") or \
            line.startswith("<!--")
def parse_hunks(diff):
    hunk_headers_indexes = []
    diff_lines = diff.splitlines()
    for index, diff_line in enumerate(diff_lines):
        if diff_line.startswith("@@ -"):
            hunk_headers_indexes.append(index)
    
    hunks_text = []
    for index, hunk_line in enumerate(hunk_headers_indexes):
        if not index == len(hunk_headers_indexes) - 1:
            hunk_text = diff_lines[hunk_line:hunk_headers_indexes[index + 1]]
        else:
            hunk_text = diff_lines[hunk_line:]
        hunks_text.append(hunk_text)

    hunks = []
    for hunk_text in hunks_text:
        hunk = {}
        hunk["raw_text"] = diff

        hunk["header"] = hunk_text[0]
        old = re.search("@@ -(.*) \+", hunk["header"]).group(1).strip().split(",")
        # Ignore subprojects commit hunks, which are very rare
        if len(old) < 2:
            continue
        hunk["old_start_line"] = old[0]
        hunk["old_length"] = old[1]
        new = re.search(".*\+(.*) @@", hunk["header"]).group(1).strip().split(",")
        # Ignore very small hungs, which are very rare
        if len(new) < 2:
            continue
        hunk["new_start_line"] = new[0]
        hunk["new_length"] = new[1]
        code_context = hunk["header"][hunk["header"].rfind("@") + 2:].strip()
        hunk["code_context"] = code_context

        change_block = [(i,line) for i, line in enumerate(hunk_text) if line.startswith("+") or line.startswith("-")]
        start_index = change_block[0][0]
        end_index = change_block[-1][0]
        hunk["before_ctx"] = hunk_text[1:start_index]
        hunk["change_block"] = [el[1] for el in change_block]
        hunk["after_ctx"] = hunk_text[end_index + 1:]
        hunks.append(hunk)
        print("This is what the hunks looks like... \n", hunks)
    return hunks

def is_valid_hunk(hunk, modified_file, commit):
    # We are interested in change blocks made of added lines only
    if not all(line.startswith("+") for line in hunk["change_block"]):
        print("In validating hunk: hunk is not valid because the lines in the change block are not all added lines")
        return False

    # If the change block is made ONLY of the following invalid lines, it is invalid for the blame
    first_line_block = int(hunk["new_start_line"]) + len(hunk["before_ctx"])
    last_line_block = first_line_block + len(hunk["change_block"]) - 1
    invalid_lines = []

    ## Step 1: useless lines are invalid
    for index, line in enumerate(hunk["change_block"]):
        if is_useless_line(line[1:].strip()):
            invalid_lines.append(index + first_line_block)
            print(f"invalid line detected in commit {commit.hash} because it is useless")
    ## Step 2: if the changed method entirely fits in the changed block, then it is a new method, and its lines are all invalid
    changed_methods = modified_file.changed_methods
    for m in changed_methods:
        if first_line_block <= m.start_line and m.end_line <= last_line_block:
            invalid_lines.extend(range(m.start_line, m.end_line + 1))
            print(f"invalid line detected in commit {commit.hash} because it is a method and fits entirely in the changed block")
    ## Step 3: if a line doesn't belong to any method (constants, global variables, typedefs, etc.), it is invalid
    methods = modified_file.methods
    for i in range(first_line_block, last_line_block + 1):
        inside = False
        for m in methods:
            if m.start_line <= i and i <= m.end_line:
                inside = True
                break
        if not inside:
            invalid_lines.append(i)
            print(f"invalid line detected in commit {commit.hash} because it does not belong to any method")
            

    # Remove duplicates and sort
    invalid_lines = sorted(list(set(invalid_lines)))
    if invalid_lines == list(range(first_line_block, last_line_block + 1)):
        return False
    return True

In [11]:
def blame_context_lines(git_repo, fix_commit, fix_files):
    print("** Blaming the context lines for add-only hunks...")
    blamed_from_context_lines = {}
    for modified_file in fix_commit.modified_files:
        filepath = get_file_name(modified_file)
        # We don't consider new files because they won't blame anything
        if modified_file.change_type == ModificationType.ADD:
            print(f"In blaming context lines: modified file is a new file in commit {fix_commit.hash}")
            continue
        # Consider only the files previously appoved
        if not filepath in fix_files:
            print("In blaming context lines: file path is not in fix files")
            continue
        hunks = parse_hunks(modified_file.diff)
        blamed_hashes = set()
        for hunk in hunks:
            # Do not consider invalid hunks
            if not is_valid_hunk(hunk, modified_file, fix_commit):
                print(f"In blaming context lines: hunk is not valid in commit {fix_commit.hash}")
                continue

            # At this point, we are sure that the hunk is made of added lines only, we can safely consider the "old start line + offset" to blame the entire hunk context only
            start_line = str(hunk["old_start_line"])
            offset = "+" + hunk["old_length"]
            # If the file is not found in the previous commits, it may be due to, rare, double renaming: we ignore these cases
            try:
                blame_output = git_repo.repo.git.blame("-w", "-c", "-l", "-L", start_line + "," + offset, fix_commit.hash + "^", "--", filepath)
            except GitCommandError:
                print(GitCommandError)
                continue
            #print("In blaming context lines: About to make the blames...")
            blamed_hashes = set()
            for blame_line in blame_output.splitlines():
                blame_line_split = blame_line.split("\t")
                blamed_hash = blame_line_split[0]
                #line_number = blame_line_split[3].split(")")[0]
                code = blame_line_split[3].split(")")[1].strip()
                # Should an empty line be blamed, ignore it
                if not code:
                    continue
                blamed_hashes.add(blamed_hash)
            blamed_from_context_lines[filepath] = blamed_hashes
    #print_blames(blamed_from_context_lines)
    return blamed_from_context_lines

In [44]:
blame_context_lines(repo, commit, fix_files)

** Blaming the context lines for add-only hunks...
This is what the hunks looks like... 
 [{'raw_text': '@@ -103,7 +103,8 @@ class LogSettingsController extends Controller {\n \t */\n \tpublic function download() {\n \t\t$resp = new StreamResponse(\\OC_Log_Owncloud::getLogFilePath());\n-\t\t$resp->addHeader(\'Content-Disposition\', \'attachment; filename="owncloud.log"\');\n+\t\t$resp->addHeader(\'Content-Type\', \'application/octet-stream\');\n+\t\t$resp->addHeader(\'Content-Disposition\', \'attachment; filename="nextcloud.log"\');\n \t\treturn $resp;\n \t}\n }\n', 'header': '@@ -103,7 +103,8 @@ class LogSettingsController extends Controller {', 'old_start_line': '103', 'old_length': '7', 'new_start_line': '103', 'new_length': '8', 'code_context': 'class LogSettingsController extends Controller {', 'before_ctx': [' \t */', ' \tpublic function download() {', ' \t\t$resp = new StreamResponse(\\OC_Log_Owncloud::getLogFilePath());'], 'change_block': ['-\t\t$resp->addHeader(\'Content-Dispo

{}

# Getting CVV from every commit

## Cloning every repo

In [2]:
with open('jsons/before_cvv_cves.json', 'r') as f:
    cve_data = json.load(f)

First, we need to clone everything. Put it in a certain fixed directory.

In [7]:
# Directory where repositories will be cloned
base_dir = 'C:\\MT_dataset_repos'
already_cloned_apps = ["NextCloud"]
# Ensure base directory exists
os.makedirs(base_dir, exist_ok=True)

In [9]:
for appname, data in cve_data.items():
    if appname not in already_cloned_apps:
        gh_link = data.get('gh_link')
        if gh_link:
            #print(gh_link, " ", appname)
            # Create directory for the app
            app_dir = os.path.join(base_dir, appname)
            os.makedirs(app_dir, exist_ok=True)

            # Clone repository
            os.system(f'git clone {gh_link}.git {app_dir}')
            print(f"Cloned {appname} repository to {app_dir}")
    else:
        print(f"Skipping cloning for {appname} as it is already cloned.")

Skipping cloning for NextCloud as it is already cloned.
Cloned Bookstack repository to C:\MT_dataset_repos\Bookstack
Cloned Magento repository to C:\MT_dataset_repos\Magento
Cloned YOURLS repository to C:\MT_dataset_repos\YOURLS
Cloned Wallabag repository to C:\MT_dataset_repos\Wallabag
Cloned WooCommerce repository to C:\MT_dataset_repos\WooCommerce
Cloned OwnCloud repository to C:\MT_dataset_repos\OwnCloud
Cloned Kanboard repository to C:\MT_dataset_repos\Kanboard
Cloned Sylius repository to C:\MT_dataset_repos\Sylius
Cloned InvoiceNinja repository to C:\MT_dataset_repos\InvoiceNinja
Cloned Opencart repository to C:\MT_dataset_repos\Opencart
Cloned HumHub repository to C:\MT_dataset_repos\HumHub
Cloned Flarum repository to C:\MT_dataset_repos\Flarum
Cloned PrivateBin repository to C:\MT_dataset_repos\PrivateBin
Cloned Pixelfed repository to C:\MT_dataset_repos\Pixelfed
Cloned Moodle repository to C:\MT_dataset_repos\Moodle
Cloned Krayin repository to C:\MT_dataset_repos\Krayin
Cloned

## Go through every repo and get the CVV

In [21]:
def get_cvv(commit_hash):
    commit = repo.get_commit(commit_hash)
    fix_files = get_fix_files(commit)
    commits_blame_del_lines = blame_deleted_lines(repo, commit, fix_files)
    commits_blame_context = blame_context_lines(repo, commit, fix_files)
    # Create a set to store unique commit hashes
    unique_commits = set()                    
    # Merge commit objects from both lists into a single list
    # Create a dictionary to store unique commit objects
    unique_commits = {}
    
    # Merge commit objects from both dictionaries into a single dictionary
    for file_path, commit_hash in commits_blame_del_lines.items():
        # Check if the commit hash is unique
        if commit_hash not in unique_commits.values():
            # Add the commit to the merged dictionary
            unique_commits[file_path] = commit_hash
    
    for file_path, commit_hash in commits_blame_context.items():
        # Check if the commit hash is unique
        if commit_hash not in unique_commits.values():
            # Add the commit to the merged dictionary
            unique_commits[file_path] = commit_hash
    return unique_commits

In [ ]:
base_dir = 'C:\\MT_dataset_repos'
test_apps = ["NextCloud", "Magento"]
for appname, data in cve_data.items():
    if appname in test_apps:
        repo_dir = os.path.join(base_dir, appname)
        repo = Git(repo_dir)
        # Iterate through data entries
        for entry in data['data']:
            # Check if 'references_transformed' attribute exists
            if 'references_transformed' in entry:
                # Iterate through commit URLs                
                for commit_url in entry['references_transformed']:
                    # Extract hash (last part of the URL)
                    commit_hash = commit_url.split('/')[-1]
                    cvv = get_cvv(commit_hash, repo)
                    entry['cvv'] = cvv

** Blaming the deleted lines...
*** Blamed 1 commit(s)
** Blaming the context lines for add-only hunks...
This is what the hunks looks like... 
 [{'raw_text': '@@ -103,7 +103,8 @@ class LogSettingsController extends Controller {\n \t */\n \tpublic function download() {\n \t\t$resp = new StreamResponse(\\OC_Log_Owncloud::getLogFilePath());\n-\t\t$resp->addHeader(\'Content-Disposition\', \'attachment; filename="owncloud.log"\');\n+\t\t$resp->addHeader(\'Content-Type\', \'application/octet-stream\');\n+\t\t$resp->addHeader(\'Content-Disposition\', \'attachment; filename="nextcloud.log"\');\n \t\treturn $resp;\n \t}\n }\n', 'header': '@@ -103,7 +103,8 @@ class LogSettingsController extends Controller {', 'old_start_line': '103', 'old_length': '7', 'new_start_line': '103', 'new_length': '8', 'code_context': 'class LogSettingsController extends Controller {', 'before_ctx': [' \t */', ' \tpublic function download() {', ' \t\t$resp = new StreamResponse(\\OC_Log_Owncloud::getLogFilePath());'], 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



** Blaming the deleted lines...
*** Blamed 0 commit(s)
** Blaming the context lines for add-only hunks...
This is what the hunks looks like... 
 [{'raw_text': "@@ -75,6 +75,11 @@ abstract class BaseResponse extends Response {\n \t\t$this->setLastModified($dataResponse->getLastModified());\n \t\t$this->setCookies($dataResponse->getCookies());\n \n+\t\tif ($dataResponse->isThrottled()) {\n+\t\t\t$throttleMetadata = $dataResponse->getThrottleMetadata();\n+\t\t\t$this->throttle($throttleMetadata);\n+\t\t}\n+\n \t\tif ($format === 'json') {\n \t\t\t$this->addHeader(\n \t\t\t\t'Content-Type', 'application/json; charset=utf-8'\n", 'header': '@@ -75,6 +75,11 @@ abstract class BaseResponse extends Response {', 'old_start_line': '75', 'old_length': '6', 'new_start_line': '75', 'new_length': '11', 'code_context': 'abstract class BaseResponse extends Response {', 'before_ctx': [' \t\t$this->setLastModified($dataResponse->getLastModified());', ' \t\t$this->setCookies($dataResponse->getCookies());',